In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Introduction to the Keras Functional API

- Sequential API : one imput, one output, one 'path' within the network !

With the **Functional API** :
- multiple inputs
- multiple outputs
- multiple pathes within the network ('Network in Network', 'Residual Block', 'Inception block')
- useful when reusing pre-trained networks

### Application : image classification with MNIST 

In [ ]:
# some datasets are available directly with Keras
from keras.datasets.mnist import load_data

# train/val/test
from sklearn.model_selection import train_test_split

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, shuffle=True, stratify=y_train, test_size=0.2)

print("x_train : {} | {}".format(x_train.shape, x_train.dtype))
print("y_train : {} | {}".format(y_train.shape, y_train.dtype))
print("x_val : {} | {}".format(x_val.shape, x_val.dtype))
print("y_val : {} | {}".format(y_val.shape, y_val.dtype))
print("x_test : {} | {}".format(x_test.shape, x_test.dtype))
print("y_test : {} | {}".format(y_test.shape, y_test.dtype))
_ = plt.hist(y_train, bins=10)

In [ ]:
fig = plt.figure(figsize=(10, 10))
for id_class in range(10):
    indices = np.where(y_train==id_class)[0]
    for i in range(10):
        plt.subplot(10, 10, id_class*10 + i + 1)
        plt.imshow(x_train[indices[i]], cmap='gray')
plt.show()

In [ ]:
# minimal pre-processsing 
# add new 'dim' : channel dimension 
x_train = np.expand_dims(x_train, axis=-1).astype(np.float32) / 255.
x_val = np.expand_dims(x_val, axis=-1).astype(np.float32) / 255.
x_test = np.expand_dims(x_test, axis=-1).astype(np.float32) / 255.

In [ ]:
# one hot encoding for the labels
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train,num_classes=10)
y_val = to_categorical(y_val,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

In [ ]:
print("x_train : {} | {}".format(x_train.shape, x_train.dtype))
print("y_train : {} | {}".format(y_train.shape, y_train.dtype))
print("x_val : {} | {}".format(x_val.shape, x_val.dtype))
print("y_val : {} | {}".format(y_val.shape, y_val.dtype))
print("x_test : {} | {}".format(x_test.shape, x_test.dtype))
print("y_test : {} | {}".format(y_test.shape, y_test.dtype))

### Build a model with the Functional API

In [ ]:
# tool box

# some layers : 
from keras.layers import Input 
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization
from keras.layers import Activation, LeakyReLU
from keras.layers import MaxPool2D, AveragePooling2D, Flatten, GlobalAveragePooling2D
from keras.layers import Add, Concatenate

# here : the functional API 
from keras.models import Model

# some optimzers
from keras.optimizers import Adam, SGD, RMSprop

# some keras callbacks to monitor the training
from keras.callbacks import Callback,EarlyStopping, ReduceLROnPlateau

In [ ]:
# need to add a 'fake' dimension for the 'gray' channel
input_layer = Input(shape=(??,??,1??))

In [ ]:
# example of neural networl (beginning)
# 28x28 feature maps
# x = Conv2D(filters=8, kernel_size=3, padding='same', strides=1, use_bias=False)(input_layer)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Conv2D(filters=16, kernel_size=3, padding='same', strides=2, use_bias=False)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)


output_layer = .....

In [ ]:
model = Model(inputs= ??? , outputs=???)
# we can use 'model' as a classical Sequential) model (.fit(), .predict(), .evaluate() ) 
model.summary()

## Launch the training

In [ ]:
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
print(ReduceLROnPlateau.__doc__)

In [ ]:
# add some callbacks

early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=3, min_delta=0.)

redure_lr = ReduceLROnPlateau(monitor='val_loss', min_lr=1e-5, patience=1, verbose=1, factor=0.1)

In [ ]:
callbacks = [early_stop, redure_lr]

In [ ]:
epochs = 100
batch_size = 64
verbose = 1 # do not set to 1 !! (issue with progbar with remore notebooks)

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks,
                    shuffle=True,
                    verbose=verbose)

In [ ]:
# plt.figure(figsize=(25,10))
fig, ax_loss = plt.subplots(figsize=(25,10))

ax_acc = ax_loss.twinx()
ax_lr = ax_loss.twinx()

ax_loss.plot(history.history['loss'], 'r')
ax_loss.plot(history.history['val_loss'], 'g')
ax_loss.set_xlabel('epochs')
ax_loss.set_ylabel('loss')

ax_acc.plot(history.history['acc'], 'r')
ax_acc.plot(history.history['val_acc'], 'g')
ax_acc.set_ylabel('accuracy')

ax_lr.plot(history.history['lr'], 'b')
ax_lr.set_ylabel('learning_rate')

In [ ]:
print(model.evaluate(x_train, y_train, verbose=0))
print(model.evaluate(x_val, y_val, verbose=0))
print(model.evaluate(x_test, y_test, verbose=0))